In [3]:
import threading
import cv2
from deepface import DeepFace
import csv
from datetime import datetime

# Initialize global variables
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0
face_match = False
thread_running = False  # To prevent multiple threads from running concurrently
matched_name = None  # Store the name of the matched person

# Reference image and name (Update as needed)
reference_images = {
    "Abdullah": "face.jpg", # Example: Key is the name, value is the file path
    "Saqlain": "facees.jpg", 
}

reference_data = {}

# Load reference images
try:
    for name, path in reference_images.items():
        img = cv2.imread(path)
        if img is None:
            raise FileNotFoundError(f"Image for {name} not found: {path}")
        reference_data[name] = cv2.resize(img, (640, 480))  # Resize to match webcam resolution
except Exception as e:
    print(f"Error loading reference images: {e}")
    exit()

# Lock for thread safety
lock = threading.Lock()

# Attendance CSV file
attendance_file = "attendance.csv"

# Initialize the attendance file
try:
    with open(attendance_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Date", "Time"])  # Header row
except Exception as e:
    print(f"Error initializing attendance file: {e}")
    exit()

# Function to mark attendance
def mark_attendance(name):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
    try:
        with open(attendance_file, "a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([name, date, time])
        print(f"Attendance marked for {name} at {time} on {date}.")
    except Exception as e:
        print(f"Error marking attendance: {e}")

# Function to verify face match
def check_face(frame):
    global face_match, thread_running, matched_name
    try:
        for name, ref_img in reference_data.items():
            result = DeepFace.verify(frame, ref_img)
            print(f"Verification result for {name}: {result['verified']}")
            if result['verified']:
                with lock:  # Safely update the global variable
                    face_match = True
                    matched_name = name
                mark_attendance(name)
                break
        else:
            with lock:
                face_match = False
                matched_name = None
    except Exception as e:
        print(f"Error during face verification: {e}")
        with lock:
            face_match = False
            matched_name = None
    finally:
        thread_running = False  # Allow new threads to be spawned

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error reading from webcam. Exiting...")
            break

        # Process every 30th frame and ensure no thread is currently running
        if counter % 30 == 0 and not thread_running:
            try:
                thread_running = True  # Indicate a thread is running
                threading.Thread(target=check_face, args=(frame.copy(),), daemon=True).start()
            except Exception as e:
                print(f"Error starting thread: {e}")
                thread_running = False

        counter += 1

        # Display the result on the frame
        with lock:  # Access global variables safely
            if face_match:
                cv2.putText(frame, f"MATCH: {matched_name}", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "NO MATCH", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Video", frame)

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
finally:
    print("Releasing resources...")
    cap.release()
    cv2.destroyAllWindows()


ImportError: DLL load failed while importing _errors: The specified procedure could not be found.